### Scopes in OO Programming

* C++ style programming language provides several scopes to allow accessibility of class members.
    * private: accessible only inside the class
    * public: accessible from entire code
    * protected: accessible by class and subclasses.
    * static : class level members.

## Python Doesn't have scope Rule.

* All members are always public and accessible.

## Conventional Coding Practices.

* since python doesn't have a scope rule, we implement certain convention.
* Any member that is to be treated as **private** should begin with **underscore**
* This doesn't really make the member private,
    * it tells the users not to access it from outside.

In [2]:
class Employee:
    def __init__(self,id,name,password,salary):
        self.id= id # ok to access from outside
        self.name=name
        self._password=password # please don't access it from outside
        self._salary=salary # don't access it from outside

    def info(self):
        return f'Employee {self.id}\t{self.name}\t{self._salary}'



In [3]:
e1=Employee(1,'Sanjay','p@ss',200000)
print(e1.info())

Employee 1	Sanjay	200000


In [5]:
print(e1.id)

1


In [6]:
print(e1._password) # We shouldn't but We can access it.

p@ss


### A more private member.

* If you member should protected from external access, we may use double underscore prefix instread of a single underscore.

* accessing such information is less easier.

In [7]:
class Employee:
    def __init__(self,id,name,password,salary):
        self.id= id # ok to access from outside
        self.name=name
        self.__password=password # please don't access it from outside
        self._salary=salary # don't access it from outside

    def info(self):
        return f'Employee {self.id}\t{self.name}\t{self._salary}'
    
    def authenticate(self,password):
        return self.__password==password



In [8]:
e=Employee(1,'Prabhat','p@ss',20000)
print(e.info())

Employee 1	Prabhat	20000


In [9]:
e.authenticate("p@ss")

True

In [10]:
e.authenticate("wrong-password")

False

#### What if we try to modify the password?

In [11]:
e.__password="hacked"

In [12]:
print(e.__password)

hacked


### Looks like password has changed.

#### But the new value is being ingored bhy authenticate

* it is still working with the old value.

In [16]:
e.authenticate('hacked')

False

In [17]:
e.authenticate("p@ss")

True

### Explanation!

* python internally stored double underscored fields differently.
* i changes there name internally.
    * The name gets additional prefix of **\_ClassName**



In [32]:

class Employee:
    def __init__(self, id, name, password):
        self.id= id  # stored as self.id
        self._name=name # stored as self._name

        self.__pasword=password # stored as self._Employee__password

    def authenticate(self, password):
        return self.__password==password # internall changes to:  self._Employee__password==password.
    
    def info(self):
        return f'Employee {self.id}\t{self._name}'


In [33]:
def object_info(object):
    return [ prop for prop in dir(object) if not prop.endswith("__")]

In [34]:
e1 = Employee(1, 'Prabhat',"p@ss")

In [35]:
object_info(e1)

['_Employee__pasword', '_name', 'authenticate', 'id', 'info']

### This name change happens only in class methods.

* Now if we assign value to **e.\_\_password**, it creates a new field.

In [36]:
e1.__password='hacked'

In [37]:
object_info(e1)

['_Employee__pasword', '__password', '_name', 'authenticate', 'id', 'info']

In [38]:
e.__password

'hacked'

### The new field is not used by class internals like authenticate.

* since the new filed is not being used. its values are UNIMPORTANT.

## But now that you know it, you can brak it.

In [39]:
e._Employee__password

'hacked'

In [40]:
e._Employee__password='hacked'

In [41]:
e.authenticate('hacked')

True

### Do you see any problem in the below code?

In [42]:
e1=Employee(1,'Sanjay',"p@ss")
e2=Employee(1,'Prabhat',"p@ss")

print(e1.info())
print(e2.info())

Employee 1	Sanjay
Employee 1	Prabhat


#### Employees shouldn't have same id

* it must be unique
* instead of taking id as parameter we can auto generate it.
* we can stored the last id generated somewhere and increment it everytime I need.

## C++ satic fields.

* C++ generally allows a class level field prefixed with static.
* A single copy of this field is maintain for class.
* It can be used for sharing some information between all objects of the class.
* All objects can access this shared  information


### Python Model

* python doesn't have **static** keyword.
* But it does have a similar concept.
* You can attach a field to a class in the same way as you attach methods.
* Such fields will have only one copy that can be accessed using
    * class reference
    * self reference.

In [54]:

class Employee:
    _last_id=0 #class level field.

    def __init__(self,  name, password):
        Employee._last_id+=1 # using class reference.

        self.id= self._last_id  # accessed using self. reference.
        self._name=name # stored as self._name

        self.__pasword=password # stored as self._Employee__password


    def get_last_id(self):
        return self._last_id #using self reference
    
    def employee_count():
        return Employee._last_id # using class reference.

    def authenticate(self, password):
        return self.__password==password # internall changes to:  self._Employee__password==password.
    
    def info(self):
        return f'Employee {self.id}\t{self._name}'


In [55]:
e1=Employee('Vivek','p@ss')
e2=Employee('Sanjay','p@ss')

print(e1.info())
print(e2.info())

Employee 1	Vivek
Employee 2	Sanjay


In [56]:
print(Employee.employee_count())
print(e1.get_last_id())
print(e2.get_last_id())

2
2
2


### class level fields with self.

* A class level field can be accessed either using
    * class as reference
    * object/self as reference.

* According to best practices guidelines
    * **A class level field should always be accessed  using  class reference and shouldn't be accessed using object/self reference**
        * since it is class level, and not associated with the object, why should we use it with object?


### self with class level fields.

* Self behaves differently with class level fields when it comes to access or modify a value.

### Accessing values using self.

* When we try to say **print(self.soemthing))**
    1. python checks if **something** is defined for he object "self"
        * if yes, it is used.

    2. if **something** is not defined for the object, python checks if **something** is defined for class of the object.
        * if yes, it is used.

    3. If it is not found in class or object, ti throws an error

In [57]:
print(e1.id) # "id" is part of object e.

1


In [58]:
print(e1._last_id) # there is no e._last_id. But there is a Employee._last_id. The same is returned.

2


### setting a value for object

* when we try to write **e.something=10**
    * python attaches this value to "e" object
    * it doesn't try to see if this property is present in the class or not.

In [59]:
object_info(e1)

['_Employee__pasword',
 '_last_id',
 '_name',
 'authenticate',
 'employee_count',
 'get_last_id',
 'id',
 'info']

In [60]:
e1._last_id=100  # this value is set only for e1 not for e2 or Employee

In [61]:
print(Employee._last_id)
print(e1._last_id)
print(e2._last_id)

2
100
2


### This may cause unintended consequences

* It is strongly recommended that when using class level fields use class reference.

In [62]:
e3=Employee('Vinod',20000)
print(e3.info())

Employee 3	Vinod


In [63]:
Employee._last_id = 50
e4=Employee('Shivanshi',10000)
print(e4.info())

Employee 51	Shivanshi
